# TCP/IP
In this exercise set, a virtual instrument is made available via the TCP/IP protocol.
In order to communicate with this instrument it is not necessary to know every detail of this protocol as there is a python package available to handle the details for you. To get an idea of the TCP/IP protocol, you can take a look at this page:
https://en.wikipedia.org/wiki/Internet_protocol_suite

The important thing for this exercise set is to know how to communicate via TCP/IP using a python library.
We will be using the socket library, see:
https://docs.python.org/3/library/socket.html

This library can be imported into python with

In [1]:
import socket

To communicate with a target machine, you will need to know its IP address and the port number.
We have set up your pytohn containers to run the measurement server as well, hence the adress will be

'localhost' 

or 

'127.0.0.1'

The port number for the multimeter is 

5000

You can instantiate a socket object with

In [4]:
s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)

As a first exercise, you will need to send a single command to the multimeter server. You can look up the appropriate commands on the website above, or by using google.
Take care that you can only send and receive 'bytes' objects. You can from and to strings by using '.encode()' and '.decode()' resepctively.

For your convenience, the commands that you can send are given below:

READ:VOL - read the voltage over the resistor

READ:CUR - read the current being send through the resistor

SET:CUR - set the current through the resistor

## exercise 1
1. Open a connection to the instrument
2. Send a command to the instrument to read the voltage
3. Print the result (do not worry about the result yet)
4. Close the connection to the instrument

In [50]:
import socket,time

def connectclose():
    s.connect((HOST, PORT))
    s.close()
    
def get_voltage():
    print(HOST)
    s.connect((HOST, PORT))
    s.sendall(b'READ:VOL')
    data = s.recv(1024)
    s.close()
    return data

# variables
HOST = 'localhost'    # The remote host
PORT = 5000         # The same port as used by the server
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

connectclose()
time.sleep(1)
get_voltage()


ConnectionRefusedError: [Errno 111] Connection refused

In [48]:
import time
time.sleep(1)

In [33]:
strtosend = 'READ:VOL'
strtosend.encode()

b'READ:VOL'

In [24]:
%%writefile instrumentdriver.py
import socket

HOST = 'localhost'    # The remote host
PORT = 5000         # The same port as used by the server
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

def connectclose():
    s.connect((HOST, PORT))
#s.sendall('READ:VOL')
#data = s.recv(1024)
    s.close()

connectclose()
print('Received')

Writing instrumentdriver.py


In [28]:
import socket

HOST = 'localhost'    # The remote host
PORT = 5000         # The same port as used by the server
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
a=s.connect((HOST, PORT))

In [29]:
a

In [ ]:
%%writefile test_instrumentdriver.py

from instrumentdriver import *

def test_connection():
    assert 

In [14]:
s.encode('READ:VOL')

AttributeError: 'socket' object has no attribute 'encode'

# Making a driver
In order to making something useful, you will need to make functions to perform specific tasks on your multimeter.
Take a collegue in mind and realise that this person does not want to know about TCP/IP, or about the specific commands that he/she has to send to the device. This person will just want to call some specific python functions.

It is your task, as writer of the driver, to make this possible by writing these functions and making sure that they will be as stable as possible

## exercise 2
1. Write separate functions to set the current and read the voltage
2. Change the read function to return a number instead of a string
3. Measure the value of your resistor

In [104]:
'''
Driver for a multimeter.
'''

import socket,time


def test_connection():
    '''Test the connection
    Returns nothing if it succeeds
    '''
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((HOST, PORT))
    s.close()
    
def set_connection():
    '''Creates connection
    Run this before communicating to the multimeter
    '''
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((HOST, PORT))
    return s

def get_voltage():
    '''Measures voltage
    Return: votage (V)
    '''
    s = set_connection()
    s.sendall(b'READ:VOL')
    data = s.recv(1024)
    s.close()
    voltage = float(data.decode().split('STAT:VOL:')[-1].split('V')[0])
    return voltage

def get_current():
    '''Returns current current
    Return: current (A)
    '''
    s = set_connection()
    s.sendall(b'READ:CUR')
    data = s.recv(1024)
    s.close()
    current = float(data.decode().split('STAT:CUR:')[-1].split('A')[0])
    return current

def set_current(I):
    '''Sets current
    Input: I (A)
    '''
    s = set_connection()
    sndstr = 'SET:CUR:'+ str(I)
    s.sendall(sndstr.encode())
    data = s.recv(1024)
    s.close()
    current = float(data.decode().split('STAT:CUR:')[-1].split('A')[0])
    print('Current set to: ',current,' A')
    

# set variables for connection
HOST = 'localhost'    # The remote host
PORT = 5000         # The same port as used by the server

# test connection
test_connection() # ok if no error occurs


In [111]:
# measuring the resistance of the resitor

set_current(0.5)
V=get_voltage()
I=get_current()
resistance = V/I

print('Measured voltage: ',V,' V')
print('Resistance of the resistor: ',resistance,' Ohms')

Current set to:  0.5  A
Measured voltage:  24.116002329944244  V
Resistance of the resistor:  48.23200465988849  Ohms


In [119]:
# CHECKING THE PUBLIC KEY FOR SSH
%%bash
ls ~/.ssh
less ~/.ssh/id_rsa.pub

id_rsa
id_rsa.pub
known_hosts
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDNT1sVeLXqkPGzCyDwy8yjBCmCNq81d0ildtFdedDR71UeIf5tL7aSK3dM65dO3IyqK7GG8eT4mZ1XGk2K0NJ6E4p0NWvhJDyLofc0z2V6REAd9RIliAZnxil7nGhr9SWO33TzhOEMxO7ClZPKL4jFUusHJ2PEXdY1QCcCu1xR3O/aSYrmuneS0m+/8P1t45gPMcAJj5OrEY02Yudy6HPCtU2nlv1mdtC/1cVpHNxYTwuN3uIj7L36GgpbrJVIxVDO4EJReiYUTaMAxewvJW0afTUQsr9JA70RwJlMttaCZd+hvBZh2ssOivTYkbVyE2+p2tNFoX9iexkd29qoIV+B jovyan@a00560ebbbdd


In [120]:
%%bash
git clone git@github.com:takafumifujita/day5-driver.git

Cloning into 'day5-driver'...


In [135]:
commit_num = commit_num + 
%%bash

cp Driver_exercises.ipynb day5-driver/excersise.ipynb
cd day5-driver/
git pull
git status
git add excersise.ipynb
git status
git commit -m 'Commit'
git pull
git push

SyntaxError: invalid syntax (<ipython-input-135-0988364100f3>, line 2)

In [131]:
%%bash
cd day5-driver/
git status
git pull
#git add excersise.ipynb
#git commit -m 'First commit'
git push

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)
nothing to commit, working directory clean
Already up-to-date.


Git 2.0 from 'matching' to 'simple'. To squelch this message
and maintain the traditional behavior, use:

  git config --global push.default matching

To squelch this message and adopt the new behavior now, use:

  git config --global push.default simple

When push.default is set to 'matching', git will push local branches
to the remote branches that already exist with the same name.

Since Git 2.0, Git defaults to the more conservative 'simple'
behavior, which only pushes the current branch to the corresponding
remote branch that 'git pull' uses to update the current branch.

See 'git help config' and search for 'push.default' for further information.
(the 'simple' mode was introduced in Git 1.7.11. Use the similar mode
'current' instead of 'simple' if you sometimes use older versions of Git)

To git@github.com:takafumifujita/day5-driver.git
   2686f32..43db13c  master -> master


# Making everything more stable
As you may have noticed, the instrument can return things that you did not expect. In this exercise you need the check the output of the multimeter, to make sure that you can take many consecutive measurement points without any gaps in the data.

Note that you must make your functions stable against faults in the measurement device. Do not try to solve this problem in your data acquisition loop.

## exercise 3
1. Add error handling to your functions
2. Average the results to reduce the noise
3. Make a graph of voltage vs. current

# Making your driver into a driver
A driver is often a bit more than a collection of functions.
In most cases, instruments have a state with them and this is one of the cases where it is useful to use an object to represent your instrument.

By writing a class for your instrument, you can also have multiple instruments at the same time. By having all instruments in different (instances of) classes, it becomes easier to address every instrument individually.

In the next exercise you will connect to two different instruments, but you will only have to write a single class to do it. By making two instances of this class, you will be able to address the two multimeters in turn.

## exercise 4
(more advanced)

1. Turn all this code into a class.
2. create two instances of this class, and try to communicate to both your own multimeter, and the multimeter of your neighbour

   hint: You will need to know the ip address of your neighbour. You can find this by typing "hostname -I" in your terminal

# A more complicated instrument
We also have made a classical simulation of a quantum computer available to you. If you find this interesting you might find it useful to try to communicate with this instrument.
the address is again your local host (127.0.0.1) and the port number is 5001

The quantum computer will have 4 qubits for you to play with.
If you want the execute Shors algorithm to factor 15, you need 7 qubits, we can make those available to you if you want.

The list of commands that you can execute on the quantum computer is

OP:[X90|Y90|X45|y45]:Q[n] (Rotate around X or Y, 90 or 45 degrees)

OP:CNOT:Qn:Qm (Perform a CNOT gate)

OP:ZERO (reset the quantum computer)

OP:MEASURE (obtain a classical readout of the quantum computer, this will destroy the state!)

Here, m,n are the indices of your qubits (starting from 0)


## exercise 5
(more advanced)

Use your knownledge aquired today to control a quantum computer. This exercise can be combined with the QASM advanced project. Try to read in a QASM file and execute the corresponding algorithm on this quantum computer.